In [1]:
import cv2
import numpy as np
import sys
import matplotlib
from matplotlib import pyplot as plt
import pylab as pl

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:
def normalize_grayscale(image, beta=0.75, image_id_k = 1, A_kp=None):
    N,M = image.shape
    k = image_id_k
    
    if image_id_k > 1 and  A_kp is None :
        print("please define A_kp as this is not the first image")
    elif image_id_k == 1:
        A_kp = np.zeros((N,M))
        
    alpha = beta*(k-1)/k
    a = np.mean(image, axis=0)
    A_k = alpha*A_kp + (1-alpha)*np.tile(a,(N,1))
    image = 128*(image/A_k)
    return image, A_k

## Canny and one Gradient

In [4]:
mytrainX = []
mytrainY = np.concatenate((np.ones(1000), np.zeros(1000)),0)
A_kp=None
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    blur = cv2.bilateralFilter(img2,10,20,20)
    blur = np.uint8(blur)
    mycanny = cv2.Canny(blur,100,200)
    mycanny = mycanny.reshape(-1)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
    mygrad = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
    mygrad = mygrad.reshape(-1)
    myfiltered = np.multiply(mygrad,mycanny)
    mytrainX.append(myfiltered)
     
A_kp=None    
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img)
    img2 = np.uint8(img2)
    blur = cv2.bilateralFilter(img2,10,20,20)
    blur = np.uint8(blur)
    mycanny = cv2.Canny(blur,100,200)
    mycanny = mycanny.reshape(-1)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
    mygrad = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
    mygrad = mygrad.reshape(-1)
    myfiltered = np.multiply(mygrad,mycanny)
    mytrainX.append(myfiltered) 
mytrainX = np.asarray(mytrainX)

In [5]:
from sklearn import svm
clf = svm.SVC()
clf.fit(mytrainX, mytrainY) 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(mytrainX, mytrainY) 
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(mytrainX, mytrainY)
logistic = linear_model.LogisticRegression(C=1e5)
logistic.fit(mytrainX, mytrainY)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [7]:
svmpospred = []
logipospred = []
regrpospred = []
knnpospred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    blur = cv2.bilateralFilter(img2,10,20,20)
    blur = np.uint8(blur)
    mycanny = cv2.Canny(blur,100,200)
    mycanny = mycanny.reshape(-1)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
    mygrad = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
    mygrad = mygrad.reshape(-1)
    myfiltered = np.multiply(mygrad,mycanny)

    logipospred.append(logistic.predict([myfiltered]))
    svmpospred.append(clf.predict([myfiltered]))
    knnpospred.append(neigh.predict([myfiltered]))
    regrpospred.append(regr.predict([myfiltered]))
 
        
    
logipred=[]
svmpred = []
regrpred = []
knnpred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    blur = cv2.bilateralFilter(img2,10,20,20)
    blur = np.uint8(blur)
    mycanny = cv2.Canny(blur,100,200)
    mycanny = mycanny.reshape(-1)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
    mygrad = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
    mygrad = mygrad.reshape(-1)
    myfiltered = np.multiply(mygrad,mycanny)
    
    logipred.append(logistic.predict([myfiltered]))
    svmpred.append(clf.predict([myfiltered]))
    knnpred.append(neigh.predict([myfiltered]))
    regrpred.append(logistic.predict([myfiltered]))

In [8]:
print ('svm pos, neg')
print (sum(svmpospred)/120)
print ((120-sum(svmpred))/120)

print ('logistic pos, neg')
print (sum(logipospred)/120)
print ((120-sum(logipred))/120)

print ('knn pos, neg')
print (sum(knnpospred)/120)
print ((120-sum(knnpred))/120)


print ('linear reg pos, neg')
poscount = 0
for i in regrpospred:
    if i>0.5:
        poscount +=1
print (poscount/120)
count = 0
for i in regrpred:
    if i>0.5:
        count +=1
print ((120-count)/120)

svm pos, neg
[ 1.]
[ 0.68333333]
logistic pos, neg
[ 0.94166667]
[ 0.85]
knn pos, neg
[ 0.89166667]
[ 0.025]
linear reg pos, neg
0.4666666666666667
0.85


## 3 Gradients with different kernels

In [10]:
mytrainX = []
mytrainY = np.concatenate((np.ones(1000), np.zeros(1000)),0)
A_kp=None
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    kernel3 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)
    gradient3 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel3)
    gradient3 = gradient3.reshape(-1)

    myfiltered = np.multiply(gradient1,gradient2)
    myfiltered = np.multiply(gradient3, myfiltered)
    mytrainX.append(myfiltered)
     
A_kp=None    
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    kernel3 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)
    gradient3 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel3)
    gradient3 = gradient3.reshape(-1)

    myfiltered = np.multiply(gradient1,gradient2)
    myfiltered = np.multiply(gradient3, myfiltered)
    mytrainX.append(myfiltered) 
mytrainX = np.asarray(mytrainX)

In [11]:
from sklearn import svm
clf = svm.SVC()
clf.fit(mytrainX, mytrainY) 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(mytrainX, mytrainY) 
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(mytrainX, mytrainY)
logistic = linear_model.LogisticRegression(C=1e5)
logistic.fit(mytrainX, mytrainY)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [12]:
svmpospred = []
logipospred = []
regrpospred = []
knnpospred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    kernel3 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)
    gradient3 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel3)
    gradient3 = gradient3.reshape(-1)

    myfiltered = np.multiply(gradient1,gradient2)
    myfiltered = np.multiply(gradient3, myfiltered)
    logipospred.append(logistic.predict([myfiltered]))
    svmpospred.append(clf.predict([myfiltered]))
    knnpospred.append(neigh.predict([myfiltered]))
    regrpospred.append(regr.predict([myfiltered]))
 
        
    
logipred=[]
svmpred = []
regrpred = []
knnpred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    kernel3 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)
    gradient3 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel3)
    gradient3 = gradient3.reshape(-1)

    myfiltered = np.multiply(gradient1,gradient2)
    myfiltered = np.multiply(gradient3, myfiltered)
    
    logipred.append(logistic.predict([myfiltered]))
    svmpred.append(clf.predict([myfiltered]))
    knnpred.append(neigh.predict([myfiltered]))
    regrpred.append(logistic.predict([myfiltered]))
    
print ('svm pos, neg')
print (sum(svmpospred)/120)
print ((120-sum(svmpred))/120)

print ('logistic pos, neg')
print (sum(logipospred)/120)
print ((120-sum(logipred))/120)

print ('knn pos, neg')
print (sum(knnpospred)/120)
print ((120-sum(knnpred))/120)


print ('linear reg pos, neg')
poscount = 0
for i in regrpospred:
    if i>0.5:
        poscount +=1
print (poscount/120)
count = 0
for i in regrpred:
    if i>0.5:
        count +=1
print ((120-count)/120)

svm pos, neg
[ 1.]
[ 0.]
logistic pos, neg
[ 0.45]
[ 0.95]
knn pos, neg
[ 0.70833333]
[ 1.]
linear reg pos, neg
0.8583333333333333
0.95


## 2 gradients

In [3]:
mytrainX = []
mytrainY = np.concatenate((np.ones(1000), np.zeros(1000)),0)
A_kp=None
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)
    
    myfiltered = np.multiply(gradient1,gradient2)

    mytrainX.append(myfiltered)
     
A_kp=None    
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    
    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)
    
    myfiltered = np.multiply(gradient1,gradient2)
    mytrainX.append(myfiltered) 
mytrainX = np.asarray(mytrainX)

In [4]:
from sklearn import svm
clf = svm.SVC()
clf.fit(mytrainX, mytrainY) 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(mytrainX, mytrainY) 
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(mytrainX, mytrainY)
logistic = linear_model.LogisticRegression(C=1e5)
logistic.fit(mytrainX, mytrainY)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [5]:
svmpospred = []
logipospred = []
regrpospred = []
knnpospred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)

    myfiltered = np.multiply(gradient1,gradient2)

    logipospred.append(logistic.predict([myfiltered]))
    svmpospred.append(clf.predict([myfiltered]))
    knnpospred.append(neigh.predict([myfiltered]))
    regrpospred.append(regr.predict([myfiltered]))
 
        
    
logipred=[]
svmpred = []
regrpred = []
knnpred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

    gradient1 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel1)
    gradient1 = gradient1.reshape(-1)
    gradient2 = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel2)
    gradient2 = gradient2.reshape(-1)

    myfiltered = np.multiply(gradient1,gradient2)
    
    logipred.append(logistic.predict([myfiltered]))
    svmpred.append(clf.predict([myfiltered]))
    knnpred.append(neigh.predict([myfiltered]))
    regrpred.append(logistic.predict([myfiltered]))
    
print ('svm pos, neg')
print (sum(svmpospred)/120)
print ((120-sum(svmpred))/120)

print ('logistic pos, neg')
print (sum(logipospred)/120)
print ((120-sum(logipred))/120)

print ('knn pos, neg')
print (sum(knnpospred)/120)
print ((120-sum(knnpred))/120)


print ('linear reg pos, neg')
poscount = 0
for i in regrpospred:
    if i>0.5:
        poscount +=1
print (poscount/120)
count = 0
for i in regrpred:
    if i>0.5:
        count +=1
print ((120-count)/120)

svm pos, neg
[ 1.]
[ 0.]
logistic pos, neg
[ 0.58333333]
[ 0.975]
knn pos, neg
[ 0.65833333]
[ 1.]
linear reg pos, neg
0.8416666666666667
0.975
